In [19]:
using VariableHeightMomentumControl
using RigidBodyDynamics
using RigidBodyDynamics.OdeIntegrators
using RigidBodyTreeInspector
using StaticArrays
using DrakeVisualizer
using ValkyrieRobot
using GeometryTypes
using ColorTypes

In [20]:
# Create mechanism
val = Valkyrie()
mechanism = val.mechanism;

In [21]:
# Add sole frame
ankleRoll = first(filter(j -> j.name == "leftAnkleRoll", joints(mechanism)))
soleFrame = CartesianFrame3D("l_foot_sole")
soleToFoot = Transform3D(soleFrame, ankleRoll.frameAfter, SVector(0.067, 0., -0.09))
add_body_fixed_frame!(mechanism, soleToFoot);

In [22]:
# Reroot at the left foot and remove fixed joints
pelvis = first(filter(b -> name(b) == "pelvis", bodies(mechanism)))
pelvisToWorld = first(filter(j -> j.name == "pelvis_to_world", joints(mechanism)))
leftFoot = first(filter(b -> name(b) == "leftFoot", bodies(mechanism)))
leftFootToWorld = Joint("leftFootToWorld", Fixed{Float64}())
world = root_body(mechanism)
attach!(mechanism, world, leftFootToWorld, Transform3D(leftFootToWorld.frameBefore, default_frame(world), -translation(soleToFoot)), leftFoot)
remove_joint!(mechanism, pelvisToWorld)

In [23]:
# remove_fixed_tree_joints!(mechanism); # currently causes vis issues

In [24]:
# Create visualizer
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window(); sleep(1)
vis = Visualizer()[:valkyrie]
setgeometry!(vis, mechanism, parse_urdf(ValkyrieRobot.urdfpath(), mechanism; package_path = [ValkyrieRobot.packagepath()]))

In [25]:
# Add box to stand on
delete!(vis[:box])
box = GeometryData(HyperRectangle(Vec(-0.4, -0.08, -0.1), Vec(0.8, 0.6, 0.1)), RGBA(0.4, 0.4, 0.4, 0.8))
addgeometry!(vis[:box], box);

In [26]:
# Create controller
centroidalFrame = CartesianFrame3D("centroidal")
stanceLegPath = path(mechanism, val.pelvis, root_body(mechanism))
stanceLegJoints = collect(stanceLegPath)
positionControlledJoints = collect(filter(j -> j ∉ stanceLegJoints, joints(mechanism)))
controller = VariableHeightMomentumController(mechanism, positionControlledJoints, centroidalFrame, soleFrame, val.pelvis);

In [27]:
function passive_dynamics!(vd::AbstractArray, t, state)
    dynamics!(result, state)
    copy!(vd, result.v̇)
    copy!(sd, result.ṡ)
end

passive_dynamics! (generic function with 1 method)

In [28]:
function controlled_dynamics!(vd::AbstractArray, sd::AbstractArray, t, state)
    torques = control(controller, t, state)
    dynamics!(result, state, torques)
    copy!(vd, result.v̇)
    copy!(sd, result.ṡ)
end

controlled_dynamics! (generic function with 1 method)

In [29]:
const τ = Vector{Float64}(num_velocities(mechanism))
function damped_dynamics!(vd::AbstractArray, sd::AbstractArray, t, state)
    τ[:] = velocity(state)
    scale!(τ, -1.0)
    dynamics!(result, state, τ)
    copy!(vd, result.v̇)
    copy!(sd, result.ṡ)
end

damped_dynamics! (generic function with 1 method)

In [30]:
# Simulation
state = MechanismState{Float64}(mechanism)
result = DynamicsResult{Float64}(mechanism)
# dyn! = damped_dynamics!
dyn! = controlled_dynamics!
sink = ExpandingStorage{Float64}(10000)
# sink = DrakeVisualizerSink(vis)
integrator = MuntheKaasIntegrator(dyn!, runge_kutta_4(Float64), sink);

In [31]:
zero!(state)
θroll = 0.16
θstanceKnee = 1.
configuration(state, findjoint(mechanism, "leftAnkleRoll"))[:] = θroll
configuration(state, findjoint(mechanism, "leftHipRoll"))[:] = -θroll
configuration(state, findjoint(mechanism, "leftKneePitch"))[:] = θstanceKnee
configuration(state, findjoint(mechanism, "leftHipPitch"))[:] = -θstanceKnee
configuration(state, findjoint(mechanism, "rightHipRoll"))[:] = -0.1
configuration(state, findjoint(mechanism, "leftShoulderRoll"))[:] = -1.
configuration(state, findjoint(mechanism, "rightShoulderRoll"))[:] = 1.
configuration(state, findjoint(mechanism, "leftShoulderYaw"))[:] = 1.2
configuration(state, findjoint(mechanism, "rightShoulderYaw"))[:] = 1.2

1.2

In [32]:
xd = 1.1#1.0 + (1.4 - 1.0) * rand()
initialize_state!(state, xd, positionControlledJoints)
settransform!(vis, state)
for joint in positionControlledJoints
    set_desired_configuration!(controller, joint, configuration(state, joint))
end

In [33]:
addgeometry!(vis, mechanism, Point3D(root_frame(mechanism), 0., -0.0426075, 0.0); radius=0.02)

In [34]:
@time integrate(integrator, state, 2., 1e-3, maxRealtimeRate = Inf)

 12

In [40]:
animate(vis, mechanism, sink.ts, sink.qs; realtimerate = 0.2, fps = 150.)